## Generated figures:
* n/a

This notebook does not generate figures, but rather performs statistical operations related to fitting exponential curves to the PC density-vs-dstance data. 

In [8]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import linregress
from scipy.optimize import curve_fit

from os import sys
sys.path.append('..')
import style
import data_util
import plotting_util

FIG_DIR = "../figures/"

In [11]:
np.random.seed(42) #note: different seed used in publication figure

def exp_curve_fit(density, start_idx=2, N_bootstraps=10000, show_plot=False):
    f = lambda x, A, tau : A*np.exp(-x/tau)
    x = (np.arange(40) + .5) * .1
    popt, _ = curve_fit(f, x[start_idx:], density[start_idx:])
    A, tau = popt
    tau_bs_vals = np.zeros(N_bootstraps)
    for i in range(N_bootstraps):
        idx = np.random.choice(np.arange(start_idx, 40), (40-start_idx), replace=True)
        popt_bs, _ = curve_fit(f, x[idx], density[idx])
        A_bs, tau_bs = popt_bs
        tau_bs_vals[i] = tau_bs
    if show_plot:
        plt.hist(tau_bs_vals, bins=50)
        plt.axvline(tau, c="red")
        plt.show()
    return tau, np.std(tau_bs_vals), tau_bs_vals

density_0 = data_util.get_pc_density_for_day(0) * 100
density_5 = data_util.get_pc_density_for_day(5) * 100

tau_0, tau_se_0, tau_bs_vals_0 = exp_curve_fit(density_0)
tau_5, tau_se_5, tau_bs_vals_5 = exp_curve_fit(density_5)

print("Day 0", tau_0, tau_se_0)
print("Day 5", tau_5, tau_se_5)

delta_tau_bs_vals = tau_bs_vals_0 - tau_bs_vals_5
plt.show()
print("p =", np.sum(delta_tau_bs_vals > 0.) / len(delta_tau_bs_vals))

Day 0 0.9723782842045255 0.20120418549321728
Day 5 1.4500780376317266 0.17797163887485207
p = 0.0371
